In [1]:
import torch
from model import CNN
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, max_error
# Determine the equipment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Calibration Transfer-merge LoRA modules with pre-trained weights

In [2]:
def merge_weights(base_model_path, lora_model_path, output_dir):
    # Load pre-trained model and weights
    base_model = CNN().to(device)
    base_model.load_state_dict(torch.load(base_model_path, map_location=device))
    base_model.load_state_dict(torch.load('iRaman-08-peft_model_BN_1%.pt', map_location=device), strict=False)
    # Perform weight decay
    original_weight = base_model.Linear.weight.data
    decayed_weight = original_weight * 0.5
    base_model.Linear.weight.data = decayed_weight
    # Load LoRA fine-tuned weights
    lora_state_dict = torch.load(lora_model_path)

    # Merge weights
    for k, v in lora_state_dict.items():
        if 'lora_A' in k:
            original_key = k.replace('.lora_A', '')
            lora_a_key = k
            lora_b_key = k.replace('lora_A', 'lora_B')

            # Get LoRA matrix
            lora_a = lora_state_dict[lora_a_key]
            lora_b = lora_state_dict[lora_b_key]

            # A*B
            merged_weight = torch.matmul(lora_b, lora_a)

            # Add the merged weights to the corresponding weights of the pre-trained model
            if original_key + '.weight' in base_model.state_dict():
                base_model.state_dict()[original_key + '.weight'] += merged_weight

    # Save merged weights
    torch.save(base_model.state_dict(), output_dir)


merge_weights('ethanol-best_model_cnn_x1.pt', 'iRaman-08-lora_state_dict_1%.pt', 'iRaman-08-transfer_model_1%.pt')

C:\Users\lai jiahui\AppData\Local\Temp\ipykernel_10256\139311063.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_model.load_state_dict(torch.load(base_model_path, m

### Test the transfer model(secondary_cnn)

In [3]:
secondary_cnn = CNN().to(device)
secondary_cnn.load_state_dict(torch.load('iRaman-08-transfer_model_1%.pt', map_location=device))


X_train = np.load('ethanol-iRaman-data\\iRaman-08_spectra_train.npy')
y_train= np.load('ethanol-iRaman-data\\iRaman-ethanol_concentrations_train.npy')
    
X_val = np.load('ethanol-iRaman-data\\iRaman-08_spectra_val.npy')
y_val= np.load('ethanol-iRaman-data\\iRaman-ethanol_concentrations_val.npy')
    
X_test = np.load('ethanol-iRaman-data\\iRaman-08_spectra_test.npy')
y_test= np.load('ethanol-iRaman-data\\iRaman-ethanol_concentrations_test.npy')
X_train, X_left, y_train, y_left = train_test_split(X_train, y_train, test_size=0.99, random_state=84)


X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
y_train = torch.tensor(y_train, dtype=torch.float32, device=device)
y_val = torch.tensor(y_val, dtype=torch.float32, device=device)
y_test = torch.tensor(y_test, dtype=torch.float32, device=device)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=len(y_train), shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(y_val), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(y_test), shuffle=False)


def evaluate_model(model, loader, device):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for batch_x, batch_y in loader:
            batch_x = batch_x.unsqueeze(dim=1).to(device)
            batch_y = batch_y.to(device)
            output = model(batch_x)
            predictions.extend(output.cpu().tolist())
            actuals.extend(batch_y.cpu().tolist())
    final_r2 = r2_score(actuals, predictions)
    RMSEP = root_mean_squared_error(actuals, predictions)
    MAE = mean_absolute_error(actuals, predictions)
    MAX_ERROR = max_error(actuals, predictions)
    return final_r2, RMSEP, MAE, MAX_ERROR

print('Training set evaluation results:')
r2, rmse, mae, me = evaluate_model(secondary_cnn, train_loader, device)
print(f'R2: {r2}, RMSEP: {rmse}, MAE: {mae}, MAX_ERROR: {me}')

print('Validation set evaluation results:')
r2, rmse, mae, me = evaluate_model(secondary_cnn, val_loader, device)
print(f'R2: {r2}, RMSEP: {rmse}, MAE: {mae}, MAX_ERROR: {me}')

print('Test set evaluation results:')
r2, rmse, mae, me = evaluate_model(secondary_cnn, test_loader, device)
print(f'R2: {r2}, RMSEP: {rmse}, MAE: {mae}, MAX_ERROR: {me}')

C:\Users\lai jiahui\AppData\Local\Temp\ipykernel_10256\1676886018.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  secondary_cnn.load_state_dict(torch.load('iRaman-08-tra

Training set evaluation results:
R2: 0.9997919287315905, RMSEP: 0.0032485056863735483, MAE: 0.0024863649159669876, MAX_ERROR: 0.0068466514348983765
Validation set evaluation results:
R2: 0.9821292369855523, RMSEP: 0.033009582692838144, MAE: 0.019685755863233848, MAX_ERROR: 0.15143048763275146
Test set evaluation results:
R2: 0.9709574006572883, RMSEP: 0.037110158056809464, MAE: 0.01969669573009014, MAX_ERROR: 0.19979363679885864


### Replace matrices A and B with two new matrices to switch to the task of analyzing another spectrometer

In [4]:

def update_weights_with_new_lora(merged_model_path, original_lora_path, new_lora_path, output_dir):
    # Load merged model and weights
    merged_model = CNN().to(device)
    merged_model.load_state_dict(torch.load(merged_model_path))
    merged_model.load_state_dict(torch.load('iRaman-19-peft_model_BN_1%.pt', map_location=device), strict=False)
    # Load original LoRA fine-tuned weights
    original_lora_state_dict = torch.load(original_lora_path)
    
    # Load new LoRA fine-tuned weights
    new_lora_state_dict = torch.load(new_lora_path)

    # Subtract original LoRA weights
    for k, v in original_lora_state_dict.items():
        if 'lora_A' in k:
            original_key = k.replace('.lora_A', '')
            lora_a_key = k
            lora_b_key = k.replace('lora_A', 'lora_B')

            # Get original LoRA matrix
            original_lora_a = original_lora_state_dict[lora_a_key]
            original_lora_b = original_lora_state_dict[lora_b_key]

            # A*B
            original_merged_weight = torch.matmul(original_lora_b, original_lora_a)

            # Subtract the original merged weights from the corresponding weights of the merged model
            if original_key + '.weight' in merged_model.state_dict():
                merged_model.state_dict()[original_key + '.weight'] -= original_merged_weight
    

    # Add new LoRA weights
    for k, v in new_lora_state_dict.items():
        if 'lora_A' in k:
            original_key = k.replace('.lora_A', '')
            lora_a_key = k
            lora_b_key = k.replace('lora_A', 'lora_B')

            # Get new LoRA matrix
            new_lora_a = new_lora_state_dict[lora_a_key]
            new_lora_b = new_lora_state_dict[lora_b_key]

            # A*B
            new_merged_weight = torch.matmul(new_lora_b, new_lora_a)

            # Add the new merged weights to the corresponding weights of the merged model
            if original_key + '.weight' in merged_model.state_dict():
                merged_model.state_dict()[original_key + '.weight'] += new_merged_weight
    
    # Save updated merged weights
    torch.save(merged_model.state_dict(), output_dir)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
update_weights_with_new_lora(
    'iRaman-08-transfer_model_1%.pt', 
    'iRaman-08-lora_state_dict_1%.pt', 
    'iRaman-19-lora_state_dict_1%.pt', 
    'iRaman-19-transfer_model_1%.pt'
)

C:\Users\lai jiahui\AppData\Local\Temp\ipykernel_10256\1356931585.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  merged_model.load_state_dict(torch.load(merged_model_pa

### Test the transfer model(secondary_cnn_2)

In [5]:
secondary_cnn_2 = CNN().to(device)
secondary_cnn_2.load_state_dict(torch.load('iRaman-19-transfer_model_1%.pt', map_location=device))


X_train = np.load('ethanol-iRaman-data\\iRaman-19_spectra_train.npy')
y_train= np.load('ethanol-iRaman-data\\iRaman-ethanol_concentrations_train.npy')
    
X_val = np.load('ethanol-iRaman-data\\iRaman-19_spectra_val.npy')
y_val= np.load('ethanol-iRaman-data\\iRaman-ethanol_concentrations_val.npy')
    
X_test = np.load('ethanol-iRaman-data\\iRaman-19_spectra_test.npy')
y_test= np.load('ethanol-iRaman-data\\iRaman-ethanol_concentrations_test.npy')
X_train, X_left, y_train, y_left = train_test_split(X_train, y_train, test_size=0.99, random_state=84)


X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
y_train = torch.tensor(y_train, dtype=torch.float32, device=device)
y_val = torch.tensor(y_val, dtype=torch.float32, device=device)
y_test = torch.tensor(y_test, dtype=torch.float32, device=device)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=len(y_train), shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(y_val), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(y_test), shuffle=False)


def evaluate_model(model, loader, device):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for batch_x, batch_y in loader:
            batch_x = batch_x.unsqueeze(dim=1).to(device)
            batch_y = batch_y.to(device)
            output = model(batch_x)
            predictions.extend(output.cpu().tolist())
            actuals.extend(batch_y.cpu().tolist())
    final_r2 = r2_score(actuals, predictions)
    RMSEP = root_mean_squared_error(actuals, predictions)
    MAE = mean_absolute_error(actuals, predictions)
    MAX_ERROR = max_error(actuals, predictions)
    return final_r2, RMSEP, MAE, MAX_ERROR

print('Training set evaluation results:')
r2, rmse, mae, me = evaluate_model(secondary_cnn_2, train_loader, device)
print(f'R2: {r2}, RMSEP: {rmse}, MAE: {mae}, MAX_ERROR: {me}')

print('Validation set evaluation results:')
r2, rmse, mae, me = evaluate_model(secondary_cnn_2, val_loader, device)
print(f'R2: {r2}, RMSEP: {rmse}, MAE: {mae}, MAX_ERROR: {me}')

print('Test set evaluation results:')
r2, rmse, mae, me = evaluate_model(secondary_cnn_2, test_loader, device)
print(f'R2: {r2}, RMSEP: {rmse}, MAE: {mae}, MAX_ERROR: {me}')

C:\Users\lai jiahui\AppData\Local\Temp\ipykernel_10256\3887108431.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  secondary_cnn_2.load_state_dict(torch.load('iRaman-19-t

Training set evaluation results:
R2: 0.9999327189793089, RMSEP: 0.00184724227755923, MAE: 0.0015965327620506287, MAX_ERROR: 0.00312688946723938
Validation set evaluation results:
R2: 0.9959643571282887, RMSEP: 0.01568645650733745, MAE: 0.01100421576174321, MAX_ERROR: 0.04738231748342514
Test set evaluation results:
R2: 0.9946978089006703, RMSEP: 0.015856332728445915, MAE: 0.011483219576378663, MAX_ERROR: 0.0435032844543457
